In [1]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv1D, MaxPooling1D, Flatten, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import joblib
import numpy as np
import pandas as pd
import tensorflow as tf
import os
import glob

In [4]:
# 컬럼명 정의
column_names = ['timestamp', 'v_raw', 'c_raw', 'voltage', 'current', 'label']

# # 정상 데이터
# normal_dir = '../realtime/normal/'
# normal_files = glob.glob(os.path.join(normal_dir, '*.csv'))
# normal_dfs = [pd.read_csv(file, names=column_names, header=None) for file in normal_files]
# normal_data = pd.concat(normal_dfs, ignore_index=True)

# # 아크 데이터
# arc_dir = '../realtime/arc/'
# arc_files = glob.glob(os.path.join(arc_dir, '*.csv'))
# arc_dfs = [pd.read_csv(file, names=column_names, header=None) for file in arc_files]
# arc_data = pd.concat(arc_dfs, ignore_index=True)

# # 전체 병합 및 셔플
# train_df = pd.concat([normal_data, arc_data], ignore_index=True)
# train_df = train_df.sample(frac=1).reset_index(drop=True)
train_df = pd.read_csv('./log/0529/학습데이터/통합 데이터(45초).csv', names=column_names, header=None)
train_df = train_df.sample(frac=1).reset_index(drop=True)

In [5]:
# 파생 피처
def add_features(df):
    df['voltage_diff'] = df['voltage'].diff().fillna(0).abs()
    df['current_diff'] = df['current'].diff().fillna(0).abs()
    df['voltage_ma'] = df['voltage'].rolling(5).mean().bfill()
    df['current_ma'] = df['current'].rolling(5).mean().bfill()
    df['power'] = df['voltage'] * df['current']
    df['power_diff'] = df['power'].diff().fillna(0).abs()
    return df

df = add_features(train_df).dropna()

In [6]:
features = ['voltage', 'current', 'voltage_diff', 'current_diff',
            'voltage_ma', 'current_ma', 'power', 'power_diff']

X_raw = df[features].values
y_raw = df['label'].values.astype(int)

# 정규화
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_raw)

In [7]:
# 시퀀스 생성
def create_sequences(data, labels, seq_len=8):
    X, y = [], []
    for i in range(len(data) - seq_len):
        X.append(data[i:i+seq_len])
        y.append(labels[i+seq_len - 1])
    return np.array(X), np.array(y)

seq_len = 8
X_seq, y_seq = create_sequences(X_scaled, y_raw, seq_len)
X_seq = X_seq.reshape((X_seq.shape[0], seq_len, len(features)))

In [9]:
# 학습/검증 분할
X_train, X_val, y_train, y_val = train_test_split(X_seq, y_seq, test_size=None, stratify=y_seq, random_state=42)

In [10]:
inputs = Input(shape=(seq_len, len(features)))
x = Conv1D(64, 3, activation='relu', padding='same')(inputs)
x = MaxPooling1D(2)(x)
x = Dropout(0.3)(x)
x = Conv1D(128, 3, activation='relu', padding='same')(x)
x = MaxPooling1D(2)(x)
x = Flatten()(x)
x = Dense(64, activation='relu')(x)
x = Dropout(0.3)(x)
outputs = Dense(1, activation='sigmoid')(x)

model = Model(inputs=inputs, outputs=outputs)
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# 학습
early_stop = EarlyStopping(patience=10, restore_best_weights=True)
model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=50, batch_size=64, callbacks=[early_stop])

Epoch 1/50
21/21 ━━━━━━━━━━━━━━━━━━━━ 2s 26ms/step - accuracy: 0.7665 - loss: 0.5262 - val_accuracy: 0.8661 - val_loss: 0.4062
Epoch 2/50
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.8646 - loss: 0.4212 - val_accuracy: 0.8661 - val_loss: 0.4010
Epoch 3/50
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.8666 - loss: 0.4135 - val_accuracy: 0.8661 - val_loss: 0.4058
Epoch 4/50
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.8510 - loss: 0.4281 - val_accuracy: 0.8661 - val_loss: 0.4067
Epoch 5/50
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.8723 - loss: 0.3791 - val_accuracy: 0.8661 - val_loss: 0.4139
Epoch 6/50
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.8722 - loss: 0.3756 - val_accuracy: 0.8661 - val_loss: 0.4111
Epoch 7/50
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.8666 - loss: 0.3825 - val_accuracy: 0.8661 - val_loss: 0.4060
Epoch 8/50
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.8608 - loss: 0.3882 - val_accuracy: 0.8661 - val_loss

In [11]:
# 모델 및 스케일러 저장
model.save('./model/cnn1d_model_new.h5')
joblib.dump(scaler, './model/scaler_cnn_new.joblib')

['./model/scaler_cnn_new.joblib']

In [13]:
import tensorflow as tf

model = tf.keras.models.load_model('./model/cnn1d_model_new.h5')

converter = tf.lite.TFLiteConverter.from_keras_model(model)

# 핵심 설정: 구버전 연산자만 사용하도록 제한
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS]  

# (선택) float16 경량화도 가능
# converter.optimizations = [tf.lite.Optimize.DEFAULT]
# converter.target_spec.supported_types = [tf.float16]

tflite_model = converter.convert()

with open("./model/cnn1d_model_compatible_new.tflite", "wb") as f:
    f.write(tflite_model)


INFO:tensorflow:Assets written to: C:\Users\NGNLAB~1\AppData\Local\Temp\tmpx4x8fit3\assets


INFO:tensorflow:Assets written to: C:\Users\NGNLAB~1\AppData\Local\Temp\tmpx4x8fit3\assets


Saved artifact at 'C:\Users\NGNLAB~1\AppData\Local\Temp\tmpx4x8fit3'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 8, 8), dtype=tf.float32, name='input_layer')
Output Type:
  TensorSpec(shape=(None, 1), dtype=tf.float32, name=None)
Captures:
  2023476598160: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2023476603088: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2023466994144: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2023476609792: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2023476613664: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2023476610848: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2023476615952: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2023476613840: TensorSpec(shape=(), dtype=tf.resource, name=None)
